In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from shapely.geometry import LineString,Point
import networkx as nx

In [ ]:
# Define Geometry of Upper West Manhattan entre
from shapely import geometry

p1 = geometry.Point(-73.97677315899479, 40.7745052767513)
p2 = geometry.Point(-73.98835952571092, 40.77956196499991)
p3 = geometry.Point(-73.98184403696767, 40.78666077169789)
p4 = geometry.Point(-73.97111445307536, 40.78212846853962)
pointList = [p1, p2, p3, p4, p1]
p = geometry.Polygon([[p.x, p.y] for p in pointList])

In [ ]:
G = ox.graph_from_polygon(p, network_type="drive")
ox.routing.add_edge_speeds(G)

In [ ]:
# Get list of edges and nodes
list_nodes = list(G.nodes)
list_edges = list(G.edges)

In [ ]:
# Characteristic length of vehicle
l_caract = 4 # m/vehicle

for ed in list_edges:
    
    if 'lanes' in G.edges[ed]:
        G.edges[ed]['lanes'] = float(G.edges[ed]['lanes'][0])
    else:
        G.edges[ed]['lanes'] = 1

    G.edges[ed]['speed_mps'] = G.edges[ed]['speed_kph']/3.6
    G.edges[ed]['capacity'] = G.edges[ed]['speed_mps']/G.edges[ed]['length']
    G.edges[ed]['xMax'] = G.edges[ed]['length']*G.edges[ed]['lanes']/l_caract


In [ ]:
# Here we need to organize the map and conncet missing points
nodes,streets = ox.graph_to_gdfs(G)
streets.explore()

In [ ]:
# Create the Dual Graph
H = nx.line_graph(G)
H.add_nodes_from((node, G.edges[node]) for node in H)

In [ ]:
# Create the position of the dual graph's nodes in the middle of the street
edges = G.edges()
nodes_position = {}
c = 0
for node in H:
    c+=1
    u, v, key = node  # Extract primal edge (u, v, key)
    
    # Get the data for the edge in the primal graph
    edge_data = G.get_edge_data(u, v, key)    
    
    # Check if the edge has a 'geometry' attribute
    if 'geometry' in edge_data:
        geom = edge_data['geometry']
    else:
        # If no geometry exists, create a simple LineString between u and v
        geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])        

    bb = geom.centroid
    x = bb.xy[0][0]
    y = bb.xy[1][0]
    nodes_position[node]=(x,y)      

In [ ]:
# Take the edges of the dual graph and assign the weight and length
edges = H.edges

for ed in edges:
    H.edges[ed]['weight'] = H.nodes[ed[0]]['capacity']

In [ ]:
# Get the Geopandas object from the graph together with the bounding box
nodes,streets = ox.graph_to_gdfs(G)

In [ ]:
# Load Data of ATC and get colum names
import numpy as np
trafficVol = pd.read_csv("ATC_Manhattan.csv")
column_name = trafficVol.columns
column_name

In [ ]:
# Extract the Segments IDs
unique_segments = trafficVol.SegmentID.unique()
# I will work with a numpy matrix instead of Pandas for simplicity
matValues = trafficVol.iloc[:,0:].values

In [ ]:
# Eliminate segments that has less than nSample points:
nSample = 10
for seg in unique_segments:
    
    ids = np.argwhere(matValues[:,8]==seg)         
    len_data = len(ids)
    if(len_data<nSample):              
        matValues = np.delete(matValues,ids,axis=0)

In [ ]:
# Create again the Data Frame with eliminated rows
trafficVol = pd.DataFrame(matValues,columns=column_name)

In [ ]:
# Extract location of ATC
trafficVol["WktGeom"] = gpd.GeoSeries.from_wkt(trafficVol["WktGeom"])

In [ ]:
gdf = gpd.GeoDataFrame(trafficVol, geometry="WktGeom",crs='2263')
gdf = gdf.to_crs(4326)
gdf = gdf.clip(p)

In [ ]:
# Eliminate spurious edges and fix some issues with two way roads
list_eliminate = [37175, 146736]
for seg in list_eliminate:
    idDrop = gdf[gdf["SegmentID"]==seg].index
    gdf = gdf.drop(idDrop)
    
mapping = {(164893,"WB"):(1061531597,42443332,0),
  (34502,"SB"):(42437384,42443332,0),
  (34488,"NB"):(42442422,9177424867,0),
  (34498,"SB"):(42428634,1061531448,0),
  (37081,"NB"):(42435272,42435275,0),
  (34399,"EB"):(595245843,42424032,0),
  (34399,"WB"):(42424032,595245843,0), # Este           
  (34405,"EB"):(42424032,42431004,0),
  (34405,"WB"):(42431004,42424032,0),
  (34505,"WB"):(42428634,42431004,0),
  (37082,"WB"):(42443336,6207264908,0),
  (37088,"WB"):(42435275,42443336,0),
  (37088,"EB"):(42443336,42435275,0),# Este
  (37089,"SB"):(42432556,42435275,0),
  (37083,"SB"):(42432558,42443336,0),
  (34512,"NB"):(42432564,42442445,0),
  (37095,"SB"):(42428061,42438859,0),
  (34536,"NB"):(42438862,42428063,0),
  (34526,"NB"):(42428068,42431019,0),
  (34542,"EB"):(42422592,42431019,0)}    

mapKeys = list(mapping.keys())
mapVals = list(mapping.values())

In [ ]:
# We add the two way segments to the GeoPandas DataFrame
gdf.insert(9,"From_OSMID",1)
gdf.insert(10,"To_OSMID",1)
gdf.insert(11,"Adj_Mat_id",1)

In [ ]:
# Extract again the numpy version to better work with it
matrix = gdf.iloc[:].values

In [ ]:
for i in range(len(mapKeys)):
    ids = np.argwhere((matrix[:,8]==mapKeys[i][0]) & (matrix[:,-1] == mapKeys[i][1])) 
    matrix[ids,9] = mapVals[i][0]
    matrix[ids,10] = mapVals[i][1]

In [ ]:
# Get Node List with the added Edges
H_node_list = list(H.nodes())
H_node_list_2 = np.array(H_node_list)
H_node_list_2 = H_node_list_2[:,0:2] 

In [ ]:
for i in range(len(matrix)):
    nodeId = np.where((H_node_list_2 == matrix[i,9:11]).all(axis=1))
    matrix[i,11] = nodeId[0][0]   

In [ ]:
column_name = gdf.columns
gdf2 = pd.DataFrame(matrix,columns=column_name)

In [ ]:
# Create a dataframe with the data of each node
NodeData = pd.DataFrame.from_dict(H.nodes, orient='index')
NodeData = NodeData.reset_index()
NodeData = NodeData.rename(columns={"level_0": "from_OSMID", "level_1": "to_OSMID", "level_2":"Adj_Id"})
NodeData["Adj_Id"]=range(len(NodeData))
W = nx.adjacency_matrix(H,weight='weight').todense()
 
xMax = NodeData["xMax"]

numNodes = H.number_of_nodes()

def Laplacian_Matrix(W):
    return np.diag(np.sum(W, axis = 1)) - W.T

Lout = Laplacian_Matrix(W)

In [ ]:
df_W = pd.DataFrame(W)

In [ ]:
# Load Calibrated parameters a and b
bMatrix = np.loadtxt('parameters_b.csv',skiprows=1,delimiter=',')
aMatrix = np.loadtxt('parameters_a.csv',skiprows=1,delimiter=',')

In [ ]:

aMatrix = aMatrix[:,1:]
bMatrix = bMatrix[:,1:]
xMax = np.array(xMax)

In [ ]:
# Calculate order parameter as a function of b in the Manhattan Matrix at two different times of the day

hora = [0,12]

kVector = np.linspace(1,50,100)
orderParameter1 = np.zeros([len(kVector),2])


for uu in range(len(hora)):

    hr = hora[uu]
    alpha = aMatrix[:,hr]
    bb = bMatrix[:,hr]

    h = 1e-2

    tSimul = np.arange(0,30,h) # 
    AlphaMatrix = np.diag(alpha)

    j = 0

    Aalpha = -(Lout+AlphaMatrix)

    for k in kVector:        

        bu = k*bb
        xStar = -np.linalg.inv(Aalpha)@bu
        xJam = np.linalg.inv(AlphaMatrix)@bu 

        x = np.zeros((len(tSimul),numNodes))

        for i in range(0,len(tSimul)-1):

            nodes_occupied = np.argwhere(x[i,:]>xMax)
            Woccupied = np.copy(W)
            Woccupied[:,nodes_occupied] = 0 
            LoutOccupied = Laplacian_Matrix(Woccupied)
            x[i+1,:] = x[i,:] + h*(-(LoutOccupied+AlphaMatrix)@x[i,:]+bu)


        xFinal = np.mean(x[-500:,:],axis=0)    
        orderParameter1[j,uu]=np.linalg.norm(xFinal-xStar)/np.linalg.norm(xStar-xJam)

        print(k,orderParameter1[j,uu])

        j+=1
    

In [ ]:
# Optimization of the network
from scipy.optimize import minimize
from scipy.sparse import csr_array,csc_array,diags_array
from scipy.sparse.linalg import inv

# This is Eq. 32
def minim_func(dw,indices,A0,b,numNodes,xMax):            
    DeltaW = csr_array((dw,indices),shape=(numNodes,numNodes)).todense()    
    AW = -Laplacian_Matrix(DeltaW)    
    Ahat = AW+A0
    xEq = -np.linalg.inv(Ahat)@b
        
    return np.max(xEq-xMax)

# This is the gradient which speeds up computaitons
def grad_func(dw,indices,A0,b,numNodes,xMax):
    grad_vector = np.zeros(len(dw))    
    DeltaW = csr_array((dw,indices),shape=(numNodes,numNodes)).todense()
    AW = -Laplacian_Matrix(DeltaW)    
    Ahat = AW+A0
    AhatInv = np.linalg.inv(Ahat)   
    xEq = -AhatInv@b 
    
    iMax = np.argmax(xEq-xMax)
    
    for q in range(len(dw)):
        i = indices[0][q]
        j = indices[1][q]        
        dAHat_dwij = np.zeros((numNodes,numNodes))
        dAHat_dwij[j,i] = 1
        dAHat_dwij[i,i] = -1
        aux = AhatInv@dAHat_dwij@AhatInv
        
        grad_vector[q] = (aux@b)[iMax] 
            
    return grad_vector

In [ ]:
horaVector = range(4,24)

# Identify intervention graph
indices = W.nonzero()
# Laplacian Matrix
Lout = Laplacian_Matrix(W)

# Optimization Loop as in previous script
for uu in range(len(horaVector)):
    hora = horaVector[uu]

    b = bMatrix[:,hora]
    alpha = aMatrix[:,hora]
    AlphaMatrix = np.diag(alpha)
    Aalpha = -(Lout+AlphaMatrix)

    A0 = -(Lout+AlphaMatrix)

    n = len(indices[0])

    numIter = 4

    wTot_Vec = np.linspace(25,50,2)

    bestXGrad = np.zeros((len(wTot_Vec),numIter))
    bestSolution = np.zeros((len(wTot_Vec),n))
    lastBest = 1e6*np.ones(len(wTot_Vec))

    for outLoop in range(numIter):    
        for k in range(len(wTot_Vec)):

            dw0 = np.random.rand(n,)

            w_tot = wTot_Vec[k]        
            print('hora = ',hora,' iter = ',outLoop,' w_tot = ',w_tot)

            cons = ({'type': 'eq', 'fun': lambda x:  w_tot-np.sum(x)})
            bounds = [(0, None) for i in range(n)]

            dw0 /= np.sum(dw0)
            dw0 *= w_tot

            # SLSQP 
            resultGrad = minimize(minim_func, x0=dw0, constraints=cons,jac=grad_func, bounds=bounds,
                options={"maxiter" : 500}, method='SLSQP', args=(indices,A0,b,numNodes,xMax))

            # Optimization result
            dw0Grad = resultGrad.x

            # Reconstruct intervention graph 
            DeltaW = csr_array((dw0Grad,indices),shape=(numNodes,numNodes)).todense()  
            AW = -Laplacian_Matrix(DeltaW)
            Ahat = AW+A0
            # New equilibrium with intervention
            x1Grad = -np.linalg.inv(Ahat)@b
            bestXGrad[k,outLoop] = np.max(x1Grad-xMax)

            # Save best result across iterations
            if(np.max(x1Grad-xMax)<lastBest[k]):
                bestSolution[k,:]=dw0Grad
                lastBest[k]=np.max(x1Grad-xMax)

    # Saves info to file for further use. Uncomment to execute next part
    #filename = 'hora' + str(hora) + 'bestSolution_grad.txt'
    #np.savetxt(filename,bestSolution)

In [ ]:
# Using the best solution possible at 25 amount of resoruces check how the ciritcal 
# k shifts at different times of the day. It uses the saved files from the previous cell

indices = W.nonzero()
Lout = Laplacian_Matrix(W)

horaVector = range(24)

kSol_Interv = np.zeros(24)
kini = 9

for hr in horaVector:
    filename = 'hora' + str(hr) + 'bestSolution_grad.txt'
    dw25,dw50 = np.loadtxt(filename)     
    
    # Reconstruct intervention graph
    DeltaW = csr_array((dw25,indices),shape=(numNodes,numNodes)).todense()  
    AW = -Laplacian_Matrix(DeltaW)    
    
    b = bMatrix[:,hr]
    alpha = aMatrix[:,hr]
    AlphaMatrix = np.diag(alpha)
    A0 = -(Lout+AlphaMatrix)    
    
    Ahat = AW+A0
    
    # Critical k where transition occurs
    kSol_Interv[hr] = fsolve(equation1,kini,args=(xMax,Ahat,b,numNodes))
    kini = kSol_Interv[hr]


In [ ]:
# Generate Figure 7

from scipy.sparse import csr_array,csc_array,diags_array

# Add required hour of the day to see the best intervention
horas = [0,12]

fig, axs = plt.subplots(2,2,dpi=300)

axs[1,0].plot(hour_vector,kSol,'-o')
axs[1,0].plot(hour_vector,kSol_Interv,'-o')
axs[1,0].set_xlabel(r'Hour')
axs[1,0].set_ylabel(r'$k_c$')
axs[1,0].legend(['Original','Intervention'])


axs[0,0].plot(kVector,orderParameter0,'o',label='00:00')
axs[0,0].plot(kVector,orderParameter12,'o',label='12:00')
axs[0,0].set_xlabel(r'$k$')
axs[0,0].set_ylabel(r'$\rho_\infty$')
axs[0,0].set_xlim(1,20)
axs[0,0].set_ylim(0,0.4)
axs[0,0].legend()

indices = W.nonzero()


for counter in range(len(horas)):

    hr = horas[counter]

    filename = 'hora' + str(hr) + 'bestSolution_grad.txt'
    dw25,dw50 = np.loadtxt(filename) 

    DeltaW = csr_array((dw50,indices),shape=(numNodes,numNodes)).todense()  

    # Relative change of the intervention graph with respect to the original weight
    maskedW = np.zeros_like(DeltaW)
    for i in range(len(DeltaW)):
        for j in range(len(DeltaW)):
            if(W[i,j]>0):
                maskedW[i,j] = DeltaW[i,j]/W[i,j]

    H3 = nx.from_numpy_array(DeltaW/20,create_using=nx.DiGraph)
    widths3 = nx.get_edge_attributes(H3, 'weight')

    edge_dictionary = nx.get_edge_attributes(H3, 'weight')
    edge_weight = np.array(list(edge_dictionary.values()))
    idxEdges = np.where(edge_weight>1e-5)
    edgeWidthFinal = tuple(edge_weight[idxEdges])


    edge_dictionary2 = nx.get_edge_attributes(H, 'weight')
    edge_keys = list(edge_dictionary2.keys())
    aaa = np.array(list(edge_dictionary2.keys()),dtype=object)
    edgeListFinal = tuple(map(tuple, aaa[idxEdges]))


    streets.plot(ax=axs[counter,1],linewidth=0.5)

    nx.draw_networkx_edges(H,pos=nodes_position,node_size=2,
                           edgelist=edgeListFinal,
                           width=edgeWidthFinal,
                           edge_color='red',
                           arrowsize=2,connectionstyle="arc3,rad=0.1",ax=axs[counter,1])

    axs[counter,1].axis('off')
    axs[counter,1].set_title(str(hr)+':00')

    
resolution_value = 300
plt.tight_layout()